This demo is designed to work with the standard socket workers alice and bob from the other demos. It adds two numbers together

In [1]:
import syft as sy
import torch
from syft.mpc import spdz
from syft.core.frameworks.torch import _GeneralizedPointerTensor as gpt
from syft.core.frameworks.torch import _MPCTensor as mpct
from syft.core.frameworks import encode
from syft.core.frameworks.torch.utils import chain_print as pp
import syft.core.frameworks.torch.utils as torch_utils
import json

In [2]:
hook = sy.TorchHook(verbose=False)
me = hook.local_worker
bob = sy.VirtualWorker(id="bob",hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice",hook=hook, is_client_worker=False)
james = sy.VirtualWorker(id="james",hook=hook, is_client_worker=False)
me.is_client_worker = False

compute_nodes = [bob, alice, james]

bob.add_workers([james, alice])
alice.add_workers([james, bob])
james.add_workers([alice, bob])

In [3]:
x = sy.Variable(torch.FloatTensor([0.1, 0.2, 0.1, 0.2])).fix_precision()

In [4]:
x.decode()

Variable containing:
 0.1000
 0.2000
 0.1000
 0.2000
[syft.core.frameworks.torch.tensor.FloatTensor of size 4]

In [13]:
x = sy.Variable(torch.FloatTensor([0.1, 0.2, 0.1, 0.2])).send(bob).fix_precision()

In [14]:
z = x.get()

In [15]:
z.decode()

Variable containing:
 0.1000
 0.2000
 0.1000
 0.2000
[syft.core.frameworks.torch.tensor.FloatTensor of size 4]

Next steps

In [4]:
x = torch.FloatTensor([0.1, 0.2, 0.1, 0.2]).send(bob).fix_precision().share(alice, james).get()
y = torch.FloatTensor([1, 2, 3, 4]).send(alice).fix_precision().share(alice, james).get()

In [5]:
pp(y)

FloatTensor > _FixedPrecisionTensor > LongTensor > _MPCTensor > LongTensor > _GeneralizedPointerTensor


In [6]:
z = x + y

In [7]:
pp(z)

FloatTensor > _FixedPrecisionTensor > LongTensor > _MPCTensor > LongTensor > _GeneralizedPointerTensor


In [8]:
z.get().decode()


 1.1000
 2.2000
 3.1000
 4.2000
[syft.core.frameworks.torch.tensor.FloatTensor of size 4]

In [9]:
pp(z)

FloatTensor > _FixedPrecisionTensor > LongTensor > _LocalTensor


In [7]:
x_alice.send(alice)
x_bob.send(bob)
y_alice.send(alice)
y_bob.send(bob)

LongTensor[_PointerTensor - id:5989216192 owner:0 loc:bob id@loc:38641853002]

### GPT

In [8]:
x_pointer_tensor_dict = {alice:x_alice.child, bob:x_bob.child}
y_pointer_tensor_dict = {alice:y_alice.child, bob:y_bob.child}

In [9]:
x_gp = gpt(x_pointer_tensor_dict).on(x)
y_gp = gpt(y_pointer_tensor_dict).on(y)

### Towards MPCT

In [10]:
x_mpc = mpct(x_gp)
y_mpc = mpct(y_gp)

In [11]:
sum_mpc = x_mpc + y_mpc

In [12]:
sum_mpc.get()


 8
[syft.core.frameworks.torch.tensor.LongTensor of size 1]

In [13]:
mul_mpc = x_mpc * y_mpc

In [14]:
mul_mpc.get()


 15
[syft.core.frameworks.torch.tensor.LongTensor of size 1]